In [1]:
import time
from re import I
import wandb
import datetime
import pdb
from plotly import express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from logging import info, debug
from tkinter import W
import numpy as np
import os
import logging
import numpy as np
from tqdm import tqdm
import matplotlib
from PIL import Image
from skimage import io
import pprint
from models.utils import show_progress
import numpy as np
from logging import log, info, debug
from PIL import Image
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2 as cv
import sys
from models.models import build_resnet, build_siamese_autoencoder
from losses import DiceLoss
from metrics import DiceMetric
from matplotlib import pyplot as plt

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

sys.path.append('/home/aadi/projects/pgrad-thesis/src/models')
sys.path.append('/home/aadi/projects/pgrad-thesis/data')
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

io.use_plugin('pil')
matplotlib.use('TkAgg')

pp = pprint.PrettyPrinter(indent=4)

2022-02-09 05:45:58,272 : Line: 117 - Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Ti, compute capability 8.6
2022-02-09 05:45:58.268459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 05:45:58.271667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 05:45:58.271832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-09 05:45:58.272033: I tensorflow/stream_executor/cuda/cuda_gpu_exe

1 Physical GPUs, 1 Logical GPUs


In [2]:
def _normalize(img):
    img = tf.cast(img, tf.float32) / 255.0
    return img


def decode_grey(img):
    img = tf.io.decode_png(img, channels=1)
    return img


def decode_rgb(img):
    img = tf.io.decode_png(img, channels=3)
    return img


def process_path_rgb(fp):
    img = tf.io.read_file(fp)
    img = decode_rgb(img)
    img = _normalize(img)
    return img


def process_path_grey(fp):
    img = tf.io.read_file(fp)
    img = decode_grey(img)
    return img


def build_data_horizontal_separate(batch_size, take, buffer_size=1000):

    def train_gen(split='train', data_path='../data/'):
        path = data_path + split
        for t1, t2, l in zip(sorted(os.listdir(path+'/time1')), sorted(os.listdir(path+'/time2')), sorted(os.listdir(path+'/label'))):
            # get full paths

            t1 = process_path_rgb(f'../data/{split}/time1/' + t1)
            t2 = process_path_rgb(f'../data/{split}/time2/' + t2)
            l = process_path_grey(f'../data/{split}/label/' + l)

            yield (t1, t2), l

    def val_gen(split='val', data_path='../data/'):
        path = data_path + split
        for t1, t2, l in zip(sorted(os.listdir(path+'/time1')), sorted(os.listdir(path+'/time2')), sorted(os.listdir(path+'/label'))):
            # get full paths

            t1 = process_path_rgb(f'data/{split}/time1/' + t1)
            t2 = process_path_rgb(f'data/{split}/time2/' + t2)
            l = process_path_grey(f'data/{split}/label/' + l)

            yield (t1, t2), l

    train_ds = tf.data.Dataset.from_generator(
        train_gen, output_types=((tf.float32, tf.float32), tf.uint8))
    val_ds = tf.data.Dataset.from_generator(
        val_gen, output_types=((tf.float32, tf.float32), tf.uint8))

    train_batches = (
        train_ds
        .cache()
        .shuffle(buffer_size)
        .batch(batch_size)
        .take(take)
        .prefetch(buffer_size=tf.data.AUTOTUNE))

    val_batches = (
        val_ds
        .cache()
        .shuffle(buffer_size)
        .batch(batch_size)
        .take(take)
        .prefetch(buffer_size=tf.data.AUTOTUNE))

    return train_batches, val_batches

In [3]:
LEARNING_RATE = 1e-3
EPOCHS = 1000
BATCH_SIZE = 25
TRAIN_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_SIZE // BATCH_SIZE
TEST_SIZE = 200

train_data, val_data = build_data_horizontal_separate(
        BATCH_SIZE, take=TRAIN_SIZE)

model = build_siamese_autoencoder()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
left (InputLayer)               [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 208         left[0][0]                       
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 1040        conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 16) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [4]:
# initialize METRICS for Tracking progress
train_dice = DiceMetric()
test_dice = DiceMetric()
val_dice = DiceMetric()

# initialize Dice LOSS for training step
dice_loss = DiceLoss()
bce_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
#--------------------------------------TENSORFLOW TRAINING LOOP STEPS---------------------------------#

@tf.function
def train_step(model, input, labels, optimizer):
    """Executes one training step and returns the loss.

    This function computes the loss and gradients, and uses the latter to
    update the model's parameters.
    """
    with tf.GradientTape() as tape:
        logits = model(input, training=True)
        # Instantiating DiceLoss() is for passing into model constructor
        print(logits.shape)
        # loss = dice_loss(labels, logits)
        loss = bce_loss(labels, logits)
    info(loss)

    grad = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grad, model.trainable_variables))

    train_dice.update_state(labels, logits)
    return loss

@tf.function
def val_step(model, input, labels):
    logits = model(input, training=False)
    val_dice.update_state(labels, logits)

    loss = bce_loss(labels, logits)
    return loss

@tf.function
def test_step(model, input, labels):
    test_logits = model(input, training=False)
    test_dice.update_state(labels, test_logits)

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

# TensorBoard logging
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
val_log_dir = 'logs/gradient_tape/' + current_time + '/val'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
val_summary_writer = tf.summary.create_file_writer(val_log_dir)


In [5]:
# for (x,y) in train_data:
#     print(x)

train_data, val_data = build_data_horizontal_separate(batch_size=10, take=100)

In [ ]:
for x, y in train_data:
    print(f'Max-y: {tf.math.reduce_max(y)}, Min-y: {tf.math.reduce_min(y)}')

In [12]:
!pip install git+https://github.com/tensorflow/examples.git

import tensorflow as tf
# was trying to see what format output should be
import tensorflow_datasets as tfds

  Cloning https://github.com/tensorflow/examples.git to /tmp/pip-req-build-g8r77xmv
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/examples.git /tmp/pip-req-build-g8r77xmv
  Resolved https://github.com/tensorflow/examples.git to commit 97dd38c31090290543e1f829fbbaddd192e8cc19
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-examples: filename=tensorflow_examples-97dd38c31090290543e1f829fbbaddd192e8cc19_-py3-none-any.whl size=268468 sha256=9a1c084a83c96b881fd5f9837b937f3dde4fabcc20df03ea9a23dae2d2c67958
  Stored in directory: /tmp/pip-ephem-wheel-cache-uk95prwr/wheels/4e/f5/c2/bfe75b834c9028b2529023bf74541c46ead531b513a8010d21
Failed to build tensorflow-examples
  Running setup.py install for tensorflow-examples ... done
  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. Di

ModuleNotFoundError: No module named 'tensorflow_datasets'